<h1 style="Comic Sans MS; text-align: center; color: #FFC300">Movie Recommender Project</h1>
<h3 id="step1" style="font-family: Comic Sans MS; color: #68FF33">1. Problem statement, data collection and processing</h3>
<p style="color: pink">The dataset for this project collects part of the knowledge from the API TMDB, which contains only 5000 movies out of the total number. We need to create a movie recommendation system using a KNN model.</p>

In [22]:
import pandas as pd

movies = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_movies.csv')
credits = pd.read_csv('https://raw.githubusercontent.com/4GeeksAcademy/k-nearest-neighbors-project-tutorial/main/tmdb_5000_credits.csv')

movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [23]:
print("The size of the movies file is ", movies.shape, " and contains the following columns:")
print(movies.columns)
print("The size of the credits file is ", credits.shape, " and contains the following columns:")
print(credits.columns)

The size of the movies file is  (4803, 20)  and contains the following columns:
Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')
The size of the credits file is  (4803, 4)  and contains the following columns:
Index(['movie_id', 'title', 'cast', 'crew'], dtype='object')


<h3 id="step2" style="font-family: Comic Sans MS; color: #68FF33">2. Creation of a database</h3>

In [24]:
import sqlite3

conn = sqlite3.connect("/workspaces/KNN-Movie-recommender/data/movies_database.db")
movies.to_sql("movies_table", conn, if_exists="replace", index=False)
credits.to_sql("credits_table", conn, if_exists="replace", index=False)


4803

<p style="color: pink">We first write the contents of the movies and credits dataframes to sql database in movies_table and credits_table. If table named movies_table already exists it will be dropped and new one will be created from movies dataframe. The index is set to False means the df index will not be written to sql.</p>

In [25]:
query = """ 
SELECT *
FROM movies_table
INNER JOIN credits_table
USING (title);
"""
total_data = pd.read_sql_query(query, conn)
conn.close()

total_data.head(1)


,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


<p style="color: pink">With the above query we are able to join the two tables using the title as the key to join them. The result is a third table with all the information we need to build our recommendation system. Now we need to clean the generated table and leave only the following columns: 'movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew'</p>

In [26]:
total_data = total_data[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
total_data.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [27]:
# Check for dupliates and drop them if any
total_data = total_data.drop_duplicates()
total_data.shape

(4809, 7)

In [28]:
# Check for missing values
total_data.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [29]:
# We have 3 in 'overview' column, let's have a look
total_data[total_data['overview'].isnull()]

,movie_id,title,overview,genres,keywords,cast,crew
2658,370980,Chiamatemi Francesco - Il Papa della gente,None,"[{""id"": 18, ""name"": ""Drama""}]","[{""id"": 717, ""name"": ""pope""}, {""id"": 5565, ""na...","[{""cast_id"": 5, ""character"": ""Jorge Mario Berg...","[{""credit_id"": ""5660019ac3a36875f100252b"", ""de..."
4145,459488,"To Be Frank, Sinatra at 100",None,"[{""id"": 99, ""name"": ""Documentary""}]","[{""id"": 6027, ""name"": ""music""}, {""id"": 225822,...","[{""cast_id"": 0, ""character"": ""Narrator"", ""cred...","[{""credit_id"": ""592b25e4c3a368783e065a2f"", ""de..."
4437,292539,Food Chains,None,"[{""id"": 99, ""name"": ""Documentary""}]",[],[],"[{""credit_id"": ""5470c3b1c3a368085e000abd"", ""de..."


In [30]:
# Let''s drop them
total_data = total_data.dropna()
total_data.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

<b><span style="color: red">Handling JSON format data</span></b>
<p style="color: pink">There are some columns in JSON format, we need to transform and process them to be able to use them in our recommendation system.</p>

In [31]:
import json

# Select the 'name' attribute and replace the 'genres' and 'keywords' columns with the 'name' attribute
total_data['genres'] = total_data['genres'].apply(json.loads)
total_data['genres'] = total_data['genres'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

total_data['keywords'] = total_data['keywords'].apply(json.loads)
total_data['keywords'] = total_data['keywords'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])

# For the column 'cast' we will select the first three actors and replace the column with the 'name' attribute
total_data['cast'] = total_data['cast'].apply(json.loads)
total_data['cast'] = total_data['cast'].apply(lambda x: [i['name'] for i in x] if isinstance(x, list) else [])
total_data['cast'] = total_data['cast'].apply(lambda x: x[:3] if len(x) >=3 else x)

# For the column 'crew' we will select the director and replace the column with the 'name' attribute
total_data['crew'] = total_data['crew'].apply(json.loads)
total_data['crew'] = total_data['crew'].apply(lambda x: [i['name'] for i in x if i['job'] == 'Director'] if isinstance(x, list) else [])

# For the column 'overview' convert it to a list
total_data['overview'] = total_data['overview'].apply(lambda x: x.split())

# Show the first row of the total_data
total_data.head()

,movie_id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton]


<ul style="color: pink">
    <li>Explaining operations for 'genres' and 'keywords' columns:</li>
        <ul>
            <li>We apply the json.loads function to every row in the 'genres' column. The json.loads function is used to parse a JSON string into a Python data structure. So, this line is converting any JSON strings in the 'genres' and 'keywords' column into Python data structures.</li>
            <li>The lambda function takes an input x and checks if x is a list using the isinstance function, if x is a list we use list comprehension to create a new list containing the 'name' value from each dictionary in the list, if x is not list then we return empty list. The objective is to extract the 'name' values from the 'genres' and 'keywords' columns.</li>
        </ul>
    <li>Explaining operations for 'cast' column:</li>
        <ul>
            <li>We apply json.loads as above.</li>
            <li>Then we apply a lambda function to each row in the column following the same as in the previous lambda for 'genres' and 'keywords'.</li>
            <li>Finally we check if the length of x is greater or equal to 3 and returns the first three elements of x, if it is less than 3 it returns x as it is.</li>
        </ul>
    <li>In 'crew' column we apply the same as above but in this case we create a new list containing the 'name' of each dictionary i in x where the 'job' key of i is equal to 'Director'.</li>        
    <li>Last, we tranform each element 'overview' column from a single string into a list of words, e.g., if an element was 'This is an overview' it would be transformed as ['This', 'is', 'an', 'overview'].</li>
</ul>
<br>
<p style="color: pink">Now we will create a new column 'tags' that will contain the columns 'genres', 'keywords', 'cast', 'crew' and 'overview' in a single column:</p>

In [32]:
total_data['tags'] = total_data['overview'] + total_data['genres'] + total_data['keywords'] + total_data['cast'] + total_data['crew']
total_data['tags'] = total_data['tags'].apply(lambda x: ','.join(x).replace(',', ' '))
total_data.head()

,movie_id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...","[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[James Cameron],In the 22nd century a paraplegic Marine is di...
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...","[Johnny Depp, Orlando Bloom, Keira Knightley]",[Gore Verbinski],Captain Barbossa long believed to be dead ha...
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...","[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Sam Mendes],A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...","[Christian Bale, Michael Caine, Gary Oldman]",[Christopher Nolan],Following the death of District Attorney Harve...
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...","[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],John Carter is a war-weary former military ca...


<p style="color: pink">Above we have created a new column named 'tags' and populated with those columns. Then we apply lambda function using join() to concatenate all the elements and replace() the commas with spaces. After this, we drop all columns except the new 'tags' column:</p>

In [33]:
total_data.drop(columns=['overview', 'genres', 'keywords', 'cast', 'crew'], inplace=True)
# An example of an element transformed below:
total_data['tags'][0]

'In the 22nd century  a paraplegic Marine is dispatched to the moon Pandora on a unique mission  but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy Science Fiction culture clash future space war space colony society space travel futuristic romance space alien tribe alien planet cgi marine soldier battle love affair anti war power relations mind and soul 3d Sam Worthington Zoe Saldana Sigourney Weaver James Cameron'

In [34]:
total_data.to_csv('/workspaces/KNN-Movie-recommender/data/processed/clean_data.csv', index=False)

conn = sqlite3.connect('/workspaces/KNN-Movie-recommender/data/movies_database.db')
movies.to_sql('clean_movies_table', conn, if_exists='replace', index=False)

4803

<h3 id="step3" style="font-family: Comic Sans MS; color: #68FF33">3. Build the KNN model</h3>

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

# Load the cleaned data
total_data = pd.read_csv('/workspaces/KNN-Movie-recommender/data/processed/clean_data.csv')

# Create a CountVectorizer object
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(total_data['tags'])

# Create a NearestNeighbors object
model = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)
model.fit(count_matrix)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

<ul style="color: pink">
    <li>A CountVectorizer object is created to convert the 'tags' column into a matrix of token counts. This is a common preprocessing step when working with text data. The fit_transform() method learns the vocabulary dictionary of the 'tags' column and returns a matrix.</li>
    <li>The NearestNeighbors object is created with the cosine similarity metric, the brute force algorithm, and 6 neighbors. The fit() method is then called on this object with the count matrix as an argument. This trains the KNN model on the data. The cosine similarity metric is a measure of similarity between two non-zero vectors, and the brute force algorithm is a way to find nearest neighbors by checking the distance to every other point.</li>
    <li>In summary, we load the text data, convert it into a numerical format, and then train a KNN model on it.</li>
</ul>
<b><span style="color: red">Defining our function</span></b>
<p style="color: pink">Now we create the function that will return the 5 movies most similar to the one we enter in the title:</p>

In [36]:
def recommend(movie_name):
    movie_index = total_data[total_data['title'] == movie_name].index[0]
    distances, indices = model.kneighbors(count_matrix[movie_index], n_neighbors=6)
    recommended_movies = []
    for i in range(1, len(distances.flatten())):
        recommended_movies.append(f"Film: {total_data['title'][indices.flatten()[i]]}")
    return recommended_movies

# Test the recommendation system
recommend('The Dark Knight Rises')

['Film: The Dark Knight',
 'Film: Batman Begins',
 'Film: Batman Returns',
 'Film: The Thief and the Cobbler',
 'Film: The Work and the Glory II: American Zion']

<p style="color: pink">The function recommend takes a movie name as input and outputs a list of recommended movies as follows:</p>
<ul style="color: pink">
    <li>We find the index of the movie in total_data df that matches the input movie name.</li>
    <li>Then we use the kneighbors method of the previously treained KNN model to find the nearest neighbors to input the movie. The kneighbors takes 2 arguments: the count matrix of the input movie and the number of neighbors to return.</li>
    <li>The kneighbors return two values: <b><em>distances</em></b> and <b><em>indices</em></b>. The value <b><em>distances</em></b> is an array containing the distances of the nearest neighbors and <b><em>indices</em></b> is a an array containing the indices of the nearest neighbors in the original dataset.</li>
    <li>We initialize an empty list to store the recommended movies. It then loops over the indices of the nearest neighbors (excluding the first index because it's the index of the input movie itself), appends the movie titles to the <b><em>recommended_movies</em></b> list and returns this list.</li>
    <li>The <b><em>flatten()</em></b></li> method is used to convert the distances into an array. 
</ul>

In [37]:
recommend('The Avengers')

['Film: Avengers: Age of Ultron',
 'Film: Captain America: The Winter Soldier',
 'Film: Iron Man 2',
 'Film: Captain America: Civil War',
 'Film: The Incredible Hulk']

In [ ]:
from pickle import dump
dump(model, open('/workspaces/KNN-Movie-recommender/models/movie_recommender_model1.plk', 'wb'))
dump(model, open('/workspaces/KNN-Movie-recommender/models/movie_recommender_model1.sav', 'wb'))